# mmap : Файлы, отображаемые в памяти

Отображение файлов в памяти обеспечивает прямой доступ к данным, хранящимся в файловой системе, .за счет использования виргуальной памяти операционной системы вместо обычных функций ввода-вывода.
Такой подход позволяет улучшить производительность операций ввода-вывода, поскольку он не требует выполнения отдельных системных вызовов для каждой попытки доступа или копирования данных между буферами. Вместо этого как ядро, так и пользовательское приложение получают прямой доступ к памяти.

В зависимости от конкретных задач файлы, отображаемые в памяти, могут рассматриваться как изменяемые строки или файловые объекты. Отображенный файл поддерживает ожидаемые методы файлового API, такие как close(), flush (), read (), readline (), seek (), tell () и write (). Он также поддерживает строковый API, предоставляющий такие средства, как взятие срезов и методы наподобие find ().
В примерах этого раздела используется текстовый файл lorem.txt,в котором содержится часть полного текста Lorem Ipsum.

# lorem.txt

Lorem ipsum dolor sit amet, consectetuer adipiscing elit.
Donec egestas, enim et consectetuer ullamcorper, lectus ligula
rutrum leo, а elementum elit tortor eu quam. Duis tincidunt nisi ut
ante. Nulla facilisi. Sed tristique eros eu libero. Pellentesque vel
arcu. Vivamus purus orci, iaculis ас, suscipit sit amet, pulvinar eu,
lacus. Praesent placerat tortor sed nisl. Nunc blandit diam egestas
dui. Pellentesque habitant morbi tristique senectus et netus et
malesuada fames ас turpis egestas. Aliquam viverra fringilla
leo. Nulla feugiat augue eleifend nulla. Vivamus mauris. Vivamus sed
mauris in nibh placerat egestas. Suspendisse potenti. Mauris
massa. Ut eget velit auctor tortor blandit sollicitudin. Suspendisse
imperdiet justo.

# Чтение

Для создания файлов в памяти предназначена функция mmap (). Ее первым аргументом является дескриптор файла, получаемый c помощью метода fileno () объекта файла или функции os. open (). Ответственность за открытие файла перед вызовом функции mmap () и его закрытие, когда необходимость в нем отпадает, возлагается на вызывающий код.

Второй аргумент функции mmap () — размер в байтах той части файла, которая подлежит отображению. Если он равен 0, то отображается весь файл. Если он превышает текущий размер файла, то файл расширяется. 
(Windows не поддерживает создание отображений нулевой длины)

Необязательный именованный аргумент access поддерживается обеими платформами. Используйте константу ACCESS_READ для доступа только по чтению, константу ACCESS_WRITE — для доступа по сквозной записи (присваивание значе-
ний отображению в памяти записывается непосредственно в файл) и константу ACCESS_COPY — для доступа c правами копирования по записи (присваивание значений отображению в памяти не записывается в файл).

In [1]:
import mmap

with open('lorem.txt','r') as f:
    with mmap.mmap(f.fileno(), 0,
                   access=mmap.ACCESS_READ) as m:
        print('First 10 bytes via read :', m.read(10))
        print('First 10 bytes via slice:', m[:10])
        print('2nd 10 bytes via read:', m.read(10))

First 10 bytes via read : b'Lorem ipsu'
First 10 bytes via slice: b'Lorem ipsu'
2nd 10 bytes via read: b'm dolor si'


Указатель файла отслеживает последний байт, к которому осуществлялся доступ посредством операции среза. В данном примере указатель перемещается вперед на 10 байт после первого чтения. Затем он перемещается в начало файла посредством операции среза и вновь перемещается вперед на 10 байт по срезу.
После выполнения операции среза повторный вызов метода read () возвращает
байты 11-20 файла.

# Запись

Указатель файла отслеживает последний байт, к которому осуществлялся доступ посредством операции среза. В данном примере указатель перемещается вперед на 10 байт после первого чтения. Затем он перемещается в начало файла посредством операции среза и вновь перемещается вперед на 10 байт по срезу.
После выполнения операции среза повторный вызов метода read () возвращает
байты 11-20 файла.

In [4]:
import mmap
import shutil

#копирование файла
shutil.copyfile('lorem.txt','lorem_copy.txt')

word=b'consectetuer'
reversed=word[::-1]
print('Looking for:', word)
print('Replacing with:', reversed)

with open ('lorem_copy.txt','r+') as f:
    with mmap.mmap(f.fileno(),0) as m:
        print('Before:\n{}'.format(m.readline().rstrip()))
        m.seek(0) # Переход в начало
        
        loc = m.find(word)
        m[loc:loc+len(word)]=reversed
        m.flush()
        
        m.seek(0)
        print('After :\n{}'.format(m.readline().rstrip()))
        
        f.seek(0)
        print('File :\n{}'.format(f.readline().rstrip()))

Looking for: b'consectetuer'
Replacing with: b'reutetcesnoc'
Before:
b'Lorem ipsum dolor sit amet, consectetuer adipiscing elit.'
After :
b'Lorem ipsum dolor sit amet, reutetcesnoc adipiscing elit.'
File :
Lorem ipsum dolor sit amet, reutetcesnoc adipiscing elit.


# Режим копирования

В режиме доступа ACCESS_COPY изменения не записываются в файл на диске.

In [9]:
import mmap
import shutil

shutil.copyfile('lorem.txt','lorem_copy.txt')

word=b'consectetuer'
reversed=word[::-1]

with open('lorem_copy.txt','r') as f:
    with mmap.mmap(f.fileno(), 0,
                   access=mmap.ACCESS_COPY) as m:
        print('Memory Before:\n{}'.format(
            m.readline().rstrip()))
        print('File Before:\n{}\n'.format(
            f.readline().rstrip()))
        
        m.seek(0)
        loc=m.find(word)
        m[loc:loc + len(word)] = reversed
        
        m.seek(0)
        print('Memory After :\n{}'.format(
            m.readline().rstrip()))
        f.seek(0)
        print('File After:\n{}'.format(
            f.readline().strip()))

Memory Before:
b'Lorem ipsum dolor sit amet, consectetuer adipiscing elit.'
File Before:
Lorem ipsum dolor sit amet, consectetuer adipiscing elit.

Memory After :
b'Lorem ipsum dolor sit amet, reutetcesnoc adipiscing elit.'
File After:
Lorem ipsum dolor sit amet, consectetuer adipiscing elit.


# Регулярные выражения

Поскольку файлы в памяти могут работать подобно строкам, их можно использовать совместно c другими модулями, оперирующими строками, такими как модуль регулярных выражений. В следующем примере выполняется поиск всех предложений, в которых содержится слово “nulla”.

In [10]:
import mmap
import re

pattern = re.compile(rb'(\.\W+)?([^.]?nulla[^.]*?\.)',
                     re.DOTALL | re.IGNORECASE | re.MULTILINE)

with open('lorem.txt','r') as f:
    with mmap.mmap(f.fileno(), 0,
                   access=mmap.ACCESS_READ) as m:
        for match in pattern.findall(m):
            print(match[1].replace(b'\n',b' '))

b'Nulla facilisi.'
b'Nulla feugiat augue eleifend nulla.'


Поскольку шаблон включает две группы, значение, возвращаемое методом
findall(), представляет собой последовательность кортежей. Инструкция
print извлекает предложение, удовлетворяющее условиям поиска, и заменяет символы перевода строки пробелами, чтобы каждый результат выводился в одной строке.

# Задание

Заменить имя 'Лада' в рассказе М.М. Пришвина на его реверсированную версию.

Лада заболела. Чашка с молоком стояла возле ее носа, она отвертывалась.

Позвали меня.

- Лада, - сказал я, - надо поесть.

Она подняла голову и забила прутом. Я погладил ее. От ласки жизнь заиграла в ее глазах.

- Кушай, Лада, - повторил я и подвинул блюдце поближе.

Она протянула нос к молоку и залакала.

Значит, через мою ласку ей силы прибавилось. Может быть, именно эти несколько глотков молока спасли ее жизнь.